In [1]:
%%capture captured_output
!pip install pulp;
!pip install pyomo;
!pip install glpk;
!apt install glpk-utils;
import pulp

In [2]:
# Define the problem with an objective to maximize profit
model = pulp.LpProblem("Maximize Profit", pulp.LpMaximize)

# Define the indices for time periods, products, and plants
periods = range(1, 6)  # Time periods 1 through 5
products = ["Widgets", "Gadgets", "Flugels"]  # Types of products
plants = ["A", "B"]  # Plant identifiers

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [3]:
# Decision Variables

# Production quantities for each product, plant, and period
production = pulp.LpVariable.dicts("production", ((product, plant, period) for product in products for plant in plants for period in periods), lowBound=0, cat='Integer')

# Inventory levels for each product, plant, and period
inventory = pulp.LpVariable.dicts("inventory", ((product, plant, period) for product in products for plant in plants for period in periods), lowBound=0, cat='Integer')

# Advertising expenses for each product and period
advertising = pulp.LpVariable.dicts("advertising", ((product, period) for product in products for period in periods), lowBound=0, cat='Integer')

# Overtime hours for each product, plant, and period
overtime = pulp.LpVariable.dicts("overtime", ((product, plant, period) for product in products for plant in plants for period in periods), lowBound=0, cat='Integer')

In [4]:
# Parameter Definitions

# Labor hours required for each product at each plant
labor_hours = {"A": {"Widgets": 9.5, "Gadgets": 7.1, "Flugels": 11.1}, "B": {"Widgets": 9.1, "Gadgets": 7.8, "Flugels": 10.6}}

# Labor costs (regular and overtime) for each plant
labor_costs = {"A": {"regular": 11, "overtime": 16.5}, "B": {"regular": 11, "overtime": 16.5}}

# Regular time hours available in each plant
regular_time_hours = {"A": 2500, "B": 3800}

# Factor by which labor costs increase in periods 3-5
increase_factor = {"A": [1, 1, 1.05, 1.05, 1.05], "B": [1, 1, 1.1, 1.1, 1.1]}

# Demand for each product in each period
demand = {"Widgets": {1: 70, 2: 125, 3: 185, 4: 190, 5: 200}, "Gadgets": {1: 200, 2: 300, 3: 295, 4: 245, 5: 240}, "Flugels": {1: 140, 2: 175, 3: 205, 4: 235, 5: 230}}

# Inventory capacity for each plant
inventory_capacity = {"A": 70, "B": 50}

# Raw material availability (in pounds)
raw_material_availability = {"RawMaterial1": 140000, "RawMaterial2": 5000}

# Raw material requirements per product per plant
raw_material_requirements = {"A": {"Widgets": {"RawMaterial1": 194, "RawMaterial2": 8.6}, "Gadgets": {"RawMaterial1": 230}, "Flugels": {"RawMaterial1": 178, "RawMaterial2": 11.6}},
                             "B": {"Widgets": {"RawMaterial1": 188, "RawMaterial2": 9.2}, "Gadgets": {"RawMaterial1": 225}, "Flugels": {"RawMaterial1": 170, "RawMaterial2": 10.8}}}

# Total advertising budget
total_advertising_budget = 70000

# Advertising costs per unit for each product
adv_cost = {"Widgets": 160, "Gadgets": 120, "Flugels": 180}

# Sales prices per unit for each product
sales_prices = {"Widgets": 2490, "Gadgets": 1990, "Flugels": 2970}

# Raw material costs per unit per product per plant
raw_material_costs = {"RawMaterial1": {"A": {"Widgets": 1.25, "Gadgets": 1.25, "Flugels": 1.25}, "B": {"Widgets": 1.45, "Gadgets": 1.45, "Flugels": 1.45}},
                      "RawMaterial2": {"A": {"Widgets": 2.65, "Gadgets": 2.65, "Flugels": 2.65}, "B": {"Widgets": 2.9, "Gadgets": 2.9, "Flugels": 2.9}}}

# Inventory holding costs per unit for each product per plant
inventory_costs = {"A": {"Widgets": 7.50, "Gadgets": 5.50, "Flugels": 6.50}, "B": {"Widgets": 7.80, "Gadgets": 5.70, "Flugels": 7.00}}

# Transportation costs per unit for each product per plant
transportation_costs = {"A": {"Widgets": 6.30, "Gadgets": 4.60, "Flugels": 5.50}, "B": {"Widgets": 6.50, "Gadgets": 5.00, "Flugels": 5.70}}


In [5]:

# Objective Function and Constraints

# Labor Costs Calculation
labor_cost = pulp.lpSum([
    labor_costs[plant]["regular"] * increase_factor[plant][period-1] * ((production[(product, plant, period)] + inventory[(product, plant, period)]) * labor_hours[plant][product] - overtime[(product, plant, period)]) +
    labor_costs[plant]["overtime"] * increase_factor[plant][period-1] * overtime[(product, plant, period)]
    for product in products for plant in plants for period in periods
])

# Overtime labor constraint
for product in products:
    for plant in plants:
        for period in periods:
            excess_labor = pulp.LpVariable(f"excess_labor_{product}_{plant}_{period}", lowBound=0, cat='Continuous')
            model += excess_labor >= (production[(product, plant, period)] + inventory[(product, plant, period)]) * labor_hours[plant][product] - regular_time_hours[plant]
            model += overtime[(product, plant, period)] >= excess_labor

# Production meets demand constraint
for product in products:
    for period in periods:
        # Revised constraint to include inventory balance
        model += pulp.lpSum([production[(product, plant, period)] for plant in plants]) + (inventory[(product, plant, period-1)] if period > 1 else 0) - (inventory[(product, plant, period)] if period < 5 else 0) == demand[product][period]

# Inventory capacity constraint
for period in periods[1:5]:
    for plant in plants:
        model += pulp.lpSum([inventory[(product, plant, period)] for product in products]) <= inventory_capacity[plant]

# Raw material usage constraint
for period in periods:
    for rm in raw_material_availability:
        model += pulp.lpSum([(production[(product, plant, period)] + inventory[(product, plant, period)]) * raw_material_requirements[plant][product].get(rm, 0) for product in products for plant in plants]) <= raw_material_availability[rm]

# Labor availability constraint
for plant in plants:
    for period in periods:
        model += pulp.lpSum([(production[(product, plant, period)] + inventory[(product, plant, period)]) * labor_hours[plant][product] for product in products]) <= regular_time_hours[plant]

# Advertising budget constraint
model += pulp.lpSum([advertising[(product, period)]*adv_cost[product] for product in products for period in periods]) <= total_advertising_budget

# No initial inventory constraint
for product in products:
    for plant in plants:
        model += inventory[(product, plant, 1)] == 0

# No final inventory constraint
for product in products:
    for plant in plants:
        model += inventory[(product, plant, 5)] == 0

In [6]:
# Revenue Calculation
revenue = pulp.lpSum([sales_prices[product] * pulp.lpSum([production[(product, plant, period)] for plant in plants for period in periods]) for product in products])

# Total Cost Calculation
total_cost = labor_cost + pulp.lpSum([
    raw_material_costs[rm][plant][product] * (production[(product, plant, period)] + inventory[(product, plant, period)]) * raw_material_requirements[plant][product].get(rm, 0)
    for product in products for plant in plants for period in periods for rm in raw_material_costs
]) + pulp.lpSum([inventory_costs[plant][product] * inventory[(product, plant, period)] for product in products for plant in plants for period in periods]) + \
pulp.lpSum([transportation_costs[plant][product] * production[(product, plant, period)] for product in products for plant in plants for period in periods]) + \
pulp.lpSum([advertising[(product, period)] * adv_cost[product] for product in products for period in periods])

# Define the objective function (Maximize Profit)
model += revenue - total_cost

In [10]:
# Solve the problem
status = model.solve()

# Enhanced status check and output results
if status == pulp.LpStatusOptimal:
    print("Status: Optimal")
    # Print the optimal production quantities and total profit with rounding
    for product in products:
        for plant in plants:
            for period in periods:
                production_value = production[(product, plant, period)].varValue
                print(f"Product: {product}, Plant: {plant}, Period: {period}, Production: {round(production_value, 2)}")
    total_profit = pulp.value(model.objective)
    print(f"Total Revenue: ${pulp.value(revenue):,.2f}")
    print(f"Total Cost: ${pulp.value(total_cost):,.2f}")
    print(f"Total Profit: ${pulp.value(model.objective):,.2f}")
else:
    print("Status:", pulp.LpStatus[model.status])

Status: Optimal
Product: Widgets, Plant: A, Period: 1, Production: 70.0
Product: Widgets, Plant: A, Period: 2, Production: 38.0
Product: Widgets, Plant: A, Period: 3, Production: 42.0
Product: Widgets, Plant: A, Period: 4, Production: 80.0
Product: Widgets, Plant: A, Period: 5, Production: 83.0
Product: Widgets, Plant: B, Period: 1, Production: 0.0
Product: Widgets, Plant: B, Period: 2, Production: 87.0
Product: Widgets, Plant: B, Period: 3, Production: 143.0
Product: Widgets, Plant: B, Period: 4, Production: 110.0
Product: Widgets, Plant: B, Period: 5, Production: 117.0
Product: Gadgets, Plant: A, Period: 1, Production: 200.0
Product: Gadgets, Plant: A, Period: 2, Production: 300.0
Product: Gadgets, Plant: A, Period: 3, Production: 295.0
Product: Gadgets, Plant: A, Period: 4, Production: 245.0
Product: Gadgets, Plant: A, Period: 5, Production: 240.0
Product: Gadgets, Plant: B, Period: 1, Production: 0.0
Product: Gadgets, Plant: B, Period: 2, Production: 0.0
Product: Gadgets, Plant: B,

In [8]:
# Ensure the model is solved
if status == pulp.LpStatusOptimal:
    print("Sensitivity Analysis Report")

    # Display shadow prices (dual values) for the constraints
    print("\nShadow Prices:")
    for name, c in model.constraints.items():
        print(f"{name}: Shadow Price = {c.pi}")

    # Display reduced costs for the variables
    print("\nReduced Costs:")
    for v in model.variables():
        print(f"{v.name}: Reduced Cost = {v.dj}")


Sensitivity Analysis Report

Shadow Prices:
_C1: Shadow Price = -0.0
_C2: Shadow Price = -0.0
_C3: Shadow Price = -0.0
_C4: Shadow Price = -0.0
_C5: Shadow Price = -0.0
_C6: Shadow Price = -0.0
_C7: Shadow Price = -0.0
_C8: Shadow Price = -0.0
_C9: Shadow Price = -0.0
_C10: Shadow Price = -0.0
_C11: Shadow Price = -0.0
_C12: Shadow Price = -0.0
_C13: Shadow Price = -0.0
_C14: Shadow Price = -0.0
_C15: Shadow Price = -0.0
_C16: Shadow Price = -0.0
_C17: Shadow Price = -0.0
_C18: Shadow Price = -0.0
_C19: Shadow Price = -0.0
_C20: Shadow Price = -0.0
_C21: Shadow Price = -0.0
_C22: Shadow Price = -0.0
_C23: Shadow Price = -0.0
_C24: Shadow Price = -0.0
_C25: Shadow Price = -0.0
_C26: Shadow Price = -0.0
_C27: Shadow Price = -0.0
_C28: Shadow Price = -0.0
_C29: Shadow Price = -0.0
_C30: Shadow Price = -0.0
_C31: Shadow Price = -0.0
_C32: Shadow Price = -0.0
_C33: Shadow Price = -0.0
_C34: Shadow Price = -0.0
_C35: Shadow Price = -0.0
_C36: Shadow Price = -0.0
_C37: Shadow Price = -0.0
_C3

## Validating the Output

In [16]:
# Production data from Output
production_data = {
    "Widgets": {"A": [70.0, 38.0, 42.0, 80.0, 83.0], "B": [0.0, 87.0, 143.0, 110.0, 117.0]},
    "Gadgets": {"A": [200.0, 300.0, 295.0, 245.0, 240.0], "B": [0.0, 0.0, 0.0, 0.0, 0.0]},
    "Flugels": {"A": [37.0, 0.0, 0.0, 0.0, 0.0], "B": [103.0, 175.0, 205.0, 235.0, 230.0]}
}

# Parameters from the problem
demand = {"Widgets": {1: 70, 2: 125, 3: 185, 4: 190, 5: 200},
          "Gadgets": {1: 200, 2: 300, 3: 295, 4: 245, 5: 240},
          "Flugels": {1: 140, 2: 175, 3: 205, 4: 235, 5: 230}}
sales_prices = {"Widgets": 2490, "Gadgets": 1990, "Flugels": 2970}

# Validate if production meets demand
meets_demand = all(
    sum(production_data[product][plant][period - 1] for plant in production_data[product]) >= demand[product][period]
    for product in demand
    for period in demand[product]
)

# Compute Revenue
revenue = sum(
    sales_prices[product] * sum(sum(production_data[product][plant]) for plant in production_data[product])
    for product in production_data
)

# Assuming the total profit to be the same as in the model output:
provided_total_profit = 6203852.90

# Compute Costs and Actual Profit
total_cost = revenue - provided_total_profit
actual_profit = revenue - total_cost

# Printing results
print(f"Does the production meet demand? {meets_demand}")
print(f"Total Revenue: ${revenue:,.2f}")
print(f"Total Cost: ${total_cost:,.2f}")
print(f"Calculated Profit: ${actual_profit:,.2f}")


Does the production meet demand? True
Total Revenue: $7,389,950.00
Total Cost: $1,186,097.10
Calculated Profit: $6,203,852.90
